## Différentaition automatique.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aderdouri/EiCNAM/blob/master/Tutorials/Notebooks/computation_graphs.ipynb) [![Open In Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/aderdouri/EiCNAM/blob/master/Tutorials/Notebooks/computation_graphs.ipynb)

In [13]:
import tensorflow as tf
from tensorflow_probability import distributions as tfd
import math
import numpy as np

### Formula:
$$
z = \cos\left(a_0 + \exp(a_1)\right)\left(\sin(a_2) + \cos(a_3)\right) + (a_1)^{\frac{3}{2}} + a_3
$$


In [1]:
# Define the variables
variables = {
    "a0": tf.Variable(1.0, dtype=tf.float32),
    "a1": tf.Variable(2.0, dtype=tf.float32),
    "a2": tf.Variable(3.0, dtype=tf.float32),
    "a3": tf.Variable(4.0, dtype=tf.float32),
}

# Define the function
def func(a0, a1, a2, a3):
    return tf.cos(a0 + tf.exp(a1)) * (tf.sin(a2) + tf.cos(a3)) + tf.pow(a1, 1.5) + a3

# Calculate the gradient
with tf.GradientTape() as tape:
    tape.watch(list(variables.values()))  # Watch the variables
    z = func(*variables.values())

# Compute gradients
grads = tape.gradient(z, list(variables.values()))

# Display gradients as a dictionary
gradient_dict = {name: float(grad.numpy()) for name, grad in zip(variables.keys(), grads)}
print("Computed Gradients:")
for var, grad in gradient_dict.items():
    print(f"{var}: {grad:.6f}")


Computed Gradients:
a0: 0.440888
a1: 5.379070
a2: 0.504802
a3: 0.614102


### Black-Scholes Formula

The Black-Scholes formula for the price of a European call option is given by:

$$
C(S, t) = S\Phi(d_1) - Ke^{-r(T-t)}\Phi(d_2),
$$

where:

$$
d_1 = \frac{\ln(S/K) + \left(r + \frac{\sigma^2}{2}\right)(T-t)}{\sigma\sqrt{T-t}}, \quad
d_2 = d_1 - \sigma\sqrt{T-t}.
$$

#### Parameters:
- C(S, t)\): Call option price at time \(t\),
- \(S\): Current price of the underlying asset,
- \(K\): Strike price of the option,
- \(r\): Risk-free interest rate,
- \(\sigma\): Volatility of the underlying asset,
- \(T\): Time to maturity,
- \($\Phi$($\cdot$)\): Cumulative distribution function of the standard normal distribution.


In [2]:
# Define the Black-Scholes formula as a function
def black_scholes(S, K, r, sigma, T, option_type="call"):
    """
    S: Current stock price
    K: Strike price
    r: Risk-free rate
    sigma: Volatility
    T: Time to maturity
    option_type: "call" or "put"
    """
    d1 = (tf.math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * tf.sqrt(T))
    d2 = d1 - sigma * tf.sqrt(T)

    if option_type == "call":
        price = S * tfp.distributions.Normal(0.0, 1.0).cdf(d1) - K * tf.exp(-r * T) * tfp.distributions.Normal(0.0, 1.0).cdf(d2)
    elif option_type == "put":
        price = K * tf.exp(-r * T) * tfp.distributions.Normal(0.0, 1.0).cdf(-d2) - S * tfp.distributions.Normal(0.0, 1.0).cdf(-d1)
    else:
        raise ValueError("Invalid option_type. Choose 'call' or 'put'.")
    return price

# Parameters
S = tf.Variable(100.0)  # Stock price
K = tf.Variable(102.0)  # Strike price
r = tf.Variable(0.05)   # Risk-free rate
sigma = tf.Variable(0.2)  # Volatility
T = tf.Variable(1.0)    # Time to maturity

# Calculate the derivatives (Greeks) using TensorFlow's GradientTape
with tf.GradientTape(persistent=True) as tape:
    tape.watch([S, K, r, sigma, T])  # Watch all inputs
    price = black_scholes(S, K, r, sigma, T)

# Compute the Greeks
delta = tape.gradient(price, S)   # Sensitivity to stock price
vega = tape.gradient(price, sigma)  # Sensitivity to volatility
theta = tape.gradient(price, T)   # Sensitivity to time to maturity
rho = tape.gradient(price, r)     # Sensitivity to risk-free rate

# Display the results
print(f"Option Price: {price.numpy():.6f}")
print(f"Delta (∂C/∂S): {delta.numpy():.6f}")
print(f"Vega (∂C/∂σ): {vega.numpy():.6f}")
print(f"Theta (∂C/∂T): {theta.numpy():.6f}")
print(f"Rho (∂C/∂r): {rho.numpy():.6f}")


Option Price: 9.423363
Delta (∂C/∂S): 0.599088
Vega (∂C/∂σ): 38.657257
Theta (∂C/∂T): 6.389997
Rho (∂C/∂r): 50.485420


In [63]:
import tensorflow as tf
from tensorflow_probability import distributions as tfd

# Constants
Z_RANGE = 1e-6  # Threshold for small z

# Normal CDF function
def normal_cdf(x):
    """Compute the CDF of the standard normal distribution."""
    normal = tfd.Normal(loc=0.0, scale=1.0)
    return normal.cdf(x)

# Black-Scholes Price Function
def bs_price(forward, volatility, numeraire, strike, expiry, is_call=True):
    """
    Compute the Black-Scholes price for a European option.

    Args:
    - forward: Forward price
    - volatility: Volatility of the underlying asset
    - numeraire: Discount factor
    - strike: Strike price
    - expiry: Time to expiry
    - is_call: Boolean, True for call option, False for put option

    Returns:
    - Option price as a TensorFlow tensor
    """
    period_volatility = volatility * tf.sqrt(expiry)
    d_plus = tf.math.log(forward / strike) / period_volatility + 0.5 * period_volatility
    d_minus = d_plus - period_volatility
    omega = tf.where(is_call, 1.0, -1.0)
    n_plus = normal_cdf(omega * d_plus)
    n_minus = normal_cdf(omega * d_minus)
    price = numeraire * omega * (forward * n_plus - strike * n_minus)
    return price


# SABR Volatility Function
def sabr_volatility(forward, alpha, beta, rho, nu, strike, expiry):
    """
    Compute the SABR volatility using TensorFlow.

    Args:
    - forward: Forward price
    - alpha: Volatility of volatility
    - beta: Beta parameter
    - rho: Correlation between forward and volatility
    - nu: Volatility of volatility
    - strike: Strike price
    - expiry: Time to expiry

    Returns:
    - Volatility as a TensorFlow tensor
    """

    beta1 = 1.0 - beta
    fKbeta = tf.pow(forward * strike, 0.5 * beta1)
    logfK = tf.math.log(forward / strike)
    z = (nu / alpha) * fKbeta * logfK

    # Compute zxz based on z
    sqz = tf.sqrt(1.0 - 2.0 * rho * z + z * z)  # Stabilized
    xz = tf.math.log(tf.maximum((sqz + z - rho) / (1.0 - rho), 1e-6))  # Stabilized
    zxz_safe = z / xz
    zxz = tf.where(tf.abs(z) < Z_RANGE, 1.0 - 0.5 * z * rho, zxz_safe)

    if (tf.math.abs(z) < 1e-6):
        zxz = 1.0 - 0.5 * z * rho;
    else:
        sqz = tf.math.sqrt(1.0 - 2.0 * rho * z + z * z)
        xz = tf.math.log((sqz + z - rho) / (1.0 - rho))
        zxz = z / xz

    # Additional factors
    beta24 = beta1 * beta1 / 24.0
    beta1920 = beta1**4 / 1920.0
    logfK2 = logfK**2

    factor11 = beta24 * logfK2
    factor12 = beta1920 * logfK2**2
    num1 = 1.0 + factor11 + factor12
    factor1 = alpha / (fKbeta * num1)  # Stabilized

    factor31 = beta24 * alpha**2 / (fKbeta**2)  # Stabilized
    factor32 = 0.25 * rho * beta * nu * alpha / (fKbeta)  # Stabilized
    factor33 = (2.0 - 3.0 * rho**2) / 24.0 * nu**2
    factor3 = 1.0 + (factor31 + factor32 + factor33) * expiry

    valatility = factor1 * zxz * factor3
    return valatility

# SABR Price Function
def sabr_price(forward, alpha, beta, rho, nu, numeraire, strike, expiry, is_call=True):
    """
    Compute the price of an option using SABR model and Black-Scholes formula.

    Args:
    - forward: Forward price
    - alpha: Volatility of volatility
    - beta: Beta parameter
    - rho: Correlation
    - nu: Volatility of volatility
    - numeraire: Discount factor
    - strike: Strike price
    - expiry: Time to expiry
    - is_call: Boolean, True for call option, False for put option

    Returns:
    - Option price as a TensorFlow tensor
    """
    volatility = sabr_volatility(forward, alpha, beta, rho, nu, strike, expiry)
    price = bs_price(forward, volatility, numeraire, strike, expiry, is_call)
    return price

In [64]:
# Define parameters as variables
forward = tf.Variable(100.0, dtype=tf.float32)      # Forward price
alpha = tf.Variable(0.2, dtype=tf.float32)          # Volatility of volatility
beta = tf.Variable(0.5, dtype=tf.float32)           # Beta parameter
rho = tf.Variable(-0.3, dtype=tf.float32)           # Correlation
nu = tf.Variable(0.4, dtype=tf.float32)             # Volatility of volatility
numeraire = tf.Variable(1.0, dtype=tf.float32)      # Numeraire (discount factor)
strike = tf.Variable(100.0, dtype=tf.float32)       # Strike price
expiry = tf.Variable(1.0, dtype=tf.float32)         # Time to expiry


# Compute gradients
with tf.GradientTape(persistent=True) as tape:
    volatility = sabr_volatility(forward, alpha, beta, rho, nu, strike, expiry)
    option_price = sabr_price(forward, alpha, beta, rho, nu, numeraire, strike, expiry)

# Sensitivities
sensitivities = {
    "dPrice/dForward": tape.gradient(option_price, forward),
    "dPrice/dAlpha": tape.gradient(option_price, alpha),
    "dPrice/dBeta": tape.gradient(option_price, beta),
    "dPrice/dRho": tape.gradient(option_price, rho),
    "dPrice/dNu": tape.gradient(option_price, nu),
    "dPrice/dNumeraire": tape.gradient(option_price, numeraire),
    "dPrice/dStrike": tape.gradient(option_price, strike),
    "dPrice/dExpiry": tape.gradient(option_price, expiry),
}

# Display results
for variable, sensitivity in sensitivities.items():
    print(f"{variable}: {sensitivity.numpy()}")

dPrice/dForward: 0.5262220501899719
dPrice/dAlpha: 4.032884120941162
dPrice/dBeta: 3.713931083679199
dPrice/dRho: 0.010371970012784004
dPrice/dNu: 0.045410607010126114
dPrice/dNumeraire: 0.8068351745605469
dPrice/dStrike: -0.5221865773200989
dPrice/dExpiry: 0.4123705327510834
